In [1]:
!pip install wandb

     |████████████████████████████████| 1.8 MB 4.2 MB/s 
     |████████████████████████████████| 144 kB 48.4 MB/s 
     |████████████████████████████████| 181 kB 49.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=f1a7a909c1ee02af3b7dbc3d6aa373a78979d79ef91652074b67f0e08a63d583
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.font_manager import FontProperties
import wandb
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

%matplotlib inline

In [3]:
%%capture
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar 
!tar -xvf  'dakshina_dataset_v1.0.tar'


In [4]:
#standardising the dataset
def opening_data(data_path):
  with open(data_path) as d:
      content = d.readlines()
  sets =[] 
  for line in content:
    datset = line.split("\t")
    sets.append(datset)
  dictionary = dict([(data[1], data[0]) for data in sets])
  return dictionary
  
train_dict = opening_data("/content/dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.train.tsv")
val_dict =   opening_data("/content/dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.dev.tsv")
test_dict = opening_data("/content/dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.test.tsv")

In [5]:
#number of training set
num_train = len(train_dict.keys())
num_test = len(test_dict.keys())
num_val = len(val_dict.keys())


In [6]:
#Data preprocessing to build the rnn model
def data_preprocess(diction):
    input_texts = []
    target_texts = []
    input_characters = []
    target_characters = []

    for key in diction:
        # Store the word in the source language
        input_texts.append(key)
        # Store the word in the target language
        target_texts.append("\t"+diction[key]+"\n")
        # Add the characters to the respective character lists
        input_characters = list(set(input_characters + list(key)))
        target_characters = list(set(target_characters + list(diction[key])))

    # Sort the input characters
    input_characters = sorted(list(set(input_characters)))
    # Sort the target characters
    target_characters = target_characters + ["\t", "\n"]
    target_characters = sorted(list(set(target_characters)))

    # Number of unique tokens in the source language
    num_encoder_tokens = len(input_characters)+1
    # Number of unique tokens in the target language
    num_decoder_tokens = len(target_characters)+1

    # Max input word length
    max_encoder_seq_length = max([len(txt) for txt in input_texts]) 
    # Max output word length
    max_decoder_seq_length = max([len(txt) for txt in target_texts])
    # Map characters to numerical indices (using +1 to avoid any character being mapped to 0)
    input_char_index = dict([(ch, i + 1) for i, ch in enumerate(input_characters)])
    target_char_index = dict([(ch, i + 1) for i, ch in enumerate(target_characters)])

    return input_characters, target_characters, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length, input_char_index, target_char_index

def one_hot_coding(diction, max_encoder_seq_length, max_decoder_seq_length, num_encoder_tokens, num_decoder_tokens):
    """
    This function takes the training/validation/test dictionary as input and produces
    the one-hot encoded versions of the respective data.
    """
    input_texts = []
    target_texts = []

    for key in diction:
        # Store the word in the source language
        input_texts.append(key)
        # Store the word in the target language
        target_texts.append("\t"+diction[key]+"\n")

    K = len(diction.keys())
    encoder_input_data = np.zeros((K, max_encoder_seq_length, num_encoder_tokens), dtype="float")
    decoder_input_data = np.zeros((K, max_decoder_seq_length, num_decoder_tokens), dtype="float")
    decoder_output_data = np.zeros((K, max_decoder_seq_length, num_decoder_tokens), dtype="float")

    for i in range(K):
        source_texts = input_texts[i]
        target_text = target_texts[i]

        # One-hot encoding for the input
        for j, ch in enumerate(source_texts):
            encoder_input_data[i, j, input_char_index[ch]] = 1.0

        # One-hot encoding for the output
        for j, ch in enumerate(target_text):
            decoder_input_data[i, j, target_char_index[ch]]= 1.0
            if j >= 1:
                # The decoder output is one step ahead of the decoder input
                decoder_output_data[i, j-1, target_char_index[ch]] = 1.0

    

    return input_texts, target_texts, encoder_input_data, decoder_input_data, decoder_output_data


In [7]:
input_characters, target_characters, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length, input_char_index, target_char_index = data_preprocess(train_dict)
input_words, target_words, encoder_input_data, decoder_input_data, decoder_output_data = one_hot_coding(train_dict, max_encoder_seq_length, max_decoder_seq_length, num_encoder_tokens, num_decoder_tokens)
val_input_words, val_target_words, val_encoder_input_data, val_decoder_input_data, val_decoder_output_data = one_hot_coding(val_dict, max_encoder_seq_length, max_decoder_seq_length, num_encoder_tokens, num_decoder_tokens)
test_input_words, test_target_words, test_encoder_input_data, test_decoder_input_data, test_decoder_output_data = one_hot_coding(test_dict, max_encoder_seq_length, max_decoder_seq_length, num_encoder_tokens, num_decoder_tokens)

In [8]:

# Dictionaries mapping from indices to characters
reverse_input_char_index = dict((i, char) for char, i in input_char_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_char_index.items())
reverse_target_char_index[0] = "\n"

In [9]:
# Using label encoding for the encoder inputs (and then find an embedding using the Embedding layer)
encoder_input_data = np.argmax(encoder_input_data, axis=2)
val_encoder_input_data = np.argmax(val_encoder_input_data, axis=2)
test_encoder_input_data = np.argmax(test_encoder_input_data, axis=2)

decoder_input_data = np.argmax(decoder_input_data, axis=2)
val_decoder_input_data = np.argmax(val_decoder_input_data, axis=2)
test_decoder_input_data = np.argmax(test_decoder_input_data, axis=2)


In [10]:
class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]


In [11]:
def build_model(latent_dim, rnn_type, embedding_dim, dropout):
   
    # ENCODER
    encoder_input = keras.Input(shape=(None, ), name="EncoderInput")
    encoder_embedding = keras.layers.Embedding(num_encoder_tokens, embedding_dim, name="EncoderInputEmbedding", mask_zero=True)(encoder_input)

    if rnn_type == 'LSTM':
        encoder_lstm = tf.keras.layers.LSTM(latent_dim,return_sequences=True,return_state=True,name="EncoderLayer" ,dropout=dropout,recurrent_dropout=dropout)
        encoder_output, state_h, state_c = encoder_lstm(encoder_embedding)
    if rnn_type == 'GRU':
        encoder_gru = tf.keras.layers.GRU(latent_dim,return_sequences=True,return_state=True,name="EncoderLayer" ,dropout=dropout,recurrent_dropout=dropout)
        encoder_output, gru_state = encoder_gru(encoder_embedding)
    if rnn_type == 'RNN':
        encoder_rnn = tf.keras.layers.SimpleRNN(latent_dim,return_sequences=True,return_state=True,name="EncoderLayer" ,dropout=dropout,recurrent_dropout=dropout)
        encoder_output, rnn_state = encoder_rnn(encoder_embedding)   

    ## DECODER
    decoder_input = keras.Input(shape=(None, ), name="DecoderInput")
    dec_emb = keras.layers.Embedding(num_decoder_tokens , 64, name="DecoderInputEmbedding", mask_zero=True)(decoder_input)

    if rnn_type == 'LSTM':
        decoder_lstm = tf.keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True, name="DecoderLayer" ,dropout=dropout,recurrent_dropout=dropout )
        decoder_output,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])
    if rnn_type == 'GRU':
        decoder_gru = tf.keras.layers.GRU(latent_dim,return_sequences=True,return_state=True,name="DecoderLayer" ,dropout=dropout,recurrent_dropout=dropout)
        decoder_output, decoder_state_gru = decoder_gru(dec_emb,initial_state = gru_state)
    if rnn_type =='RNN':
        decoder_rnn = tf.keras.layers.SimpleRNN(latent_dim,return_sequences=True,return_state=True,name="DecoderLayer" ,dropout=dropout,recurrent_dropout=dropout)
        decoder_output, decoder_state_rnn = decoder_rnn(dec_emb,initial_state = rnn_state)

    # Attention layer
    attn_out, attn_states = AttentionLayer(name='attention_layer')([encoder_output, decoder_output])


    # Concat attention input and decoder LSTM output
    decoder_concat_input = tf.keras.layers.Concatenate(axis=-1, name='concat_layer')([decoder_output, attn_out])

    #dense layer
    decoder_dense =  tf.keras.layers.TimeDistributed(keras.layers.Dense(num_decoder_tokens, activation='softmax'))
    decoder_outputs = decoder_dense(decoder_concat_input)

    # Define the model 
    model = keras.Model([encoder_input, decoder_input], decoder_outputs)

    return model


In [12]:
def build_inference_model(model,rnn_type, latent_dim):
  if rnn_type == "LSTM":
              # Input to the encoder
              encoder_inputs = model.input[0]

              # Output of the encoder
              encoder_outputs, state_h_enc, state_c_enc = model.layers[4].output
              encoder_states = [encoder_outputs,state_h_enc, state_c_enc]

              # Create an encoder model 
              encoder_model = keras.Model(encoder_inputs, encoder_states)


              # Input to the decoder

              decoder_state_input_h = keras.Input(shape=(latent_dim,))
              decoder_state_input_c = keras.Input(shape=(latent_dim,))
              decoder_hidden_state_inputs = keras.Input(shape=(None,latent_dim))
              decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

              decoder_inputs = model.layers[1].output
              decoder_embedding_layer = model.layers[3]
              decoder_embedding = decoder_embedding_layer(decoder_inputs)

              decoder_lstm = model.layers[5]

              decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
              decoder_states = [state_h_dec, state_c_dec]

              #attention
              attn_layer = model.layers[6]
              attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_inputs,decoder_outputs])

              #concat
              concate = model.layers[7]
              decoder_inf_concat = concate([decoder_outputs,attn_out_inf])

              # Softmax layer
              decoder_dense = model.layers[8]
              decoder_outputs = decoder_dense(decoder_inf_concat)

              # Create the decoder model
              decoder_model = keras.Model([decoder_inputs] + [decoder_hidden_state_inputs,decoder_state_input_h,decoder_state_input_c], [decoder_outputs] + decoder_states +[attn_states_inf])


  else:
              # Input to the encoder
              encoder_inputs = model.input[0]

              # Output of the encoder
              encoder_outputs, state_gru_enc = model.layers[4].output
              encoder_states = [encoder_outputs,state_gru_enc]

              # Create an encoder model 
              encoder_model = keras.Model(encoder_inputs, encoder_states)

              # Input to the decoder

              decoder_state_input = keras.Input(shape=(latent_dim,))
              decoder_hidden_state_inputs = keras.Input(shape=(None,latent_dim))
              decoder_states_inputs = [decoder_state_input]

              decoder_inputs = model.layers[1].output
              decoder_embedding_layer = model.layers[3]
              decoder_embedding = decoder_embedding_layer(decoder_inputs)

              decoder_gru = model.layers[5]

              decoder_outputs, state_gru_dec = decoder_gru(decoder_embedding, initial_state=decoder_states_inputs)
              decoder_states = [state_gru_dec]


              #attention
              attn_layer = model.layers[6]
              attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_inputs,decoder_outputs])

              #concat
              concate = model.layers[7]
              decoder_inf_concat = concate([decoder_outputs,attn_out_inf])

              # Softmax layer
              decoder_dense = model.layers[8]
              decoder_outputs = decoder_dense(decoder_inf_concat)

              # Create the decoder model
              decoder_model = keras.Model([decoder_inputs] + [decoder_hidden_state_inputs,decoder_state_input], [decoder_outputs] + decoder_states +[attn_states_inf])

  return encoder_model, decoder_model


In [13]:
def train():

   

    # Define the model
    model = build_model(256, "GRU", 258, 0.3)

    # Compile the model
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

    # Train the model
    history = model.fit(
            [encoder_input_data, decoder_input_data],
            decoder_output_data,
            batch_size = 64,
            epochs = 10,
            verbose = 2)
    return model
   

In [14]:
model = train()

Epoch 1/10
1036/1036 - 484s - loss: 0.3017 - accuracy: 0.6935 - 484s/epoch - 467ms/step
Epoch 2/10
1036/1036 - 470s - loss: 0.1228 - accuracy: 0.8221 - 470s/epoch - 454ms/step
Epoch 3/10
1036/1036 - 466s - loss: 0.1023 - accuracy: 0.8361 - 466s/epoch - 450ms/step
Epoch 4/10
1036/1036 - 467s - loss: 0.0894 - accuracy: 0.8446 - 467s/epoch - 451ms/step
Epoch 5/10
1036/1036 - 466s - loss: 0.0801 - accuracy: 0.8511 - 466s/epoch - 450ms/step
Epoch 6/10
1036/1036 - 461s - loss: 0.0727 - accuracy: 0.8564 - 461s/epoch - 445ms/step
Epoch 7/10
1036/1036 - 459s - loss: 0.0671 - accuracy: 0.8602 - 459s/epoch - 443ms/step
Epoch 8/10
1036/1036 - 461s - loss: 0.0619 - accuracy: 0.8637 - 461s/epoch - 445ms/step
Epoch 9/10
1036/1036 - 463s - loss: 0.0575 - accuracy: 0.8669 - 463s/epoch - 447ms/step
Epoch 10/10
1036/1036 - 463s - loss: 0.0546 - accuracy: 0.8687 - 463s/epoch - 447ms/step


In [15]:

# Get the encoder and decoder model
encoder_model, decoder_model = build_inference_model(model,"GRU",256)
def calculate_accuracy(encoder_model,decoder_model,input_data,target_words):
    predicted = []
    n = input_data.shape[0]
    batch_size = 700
    for i in range(0, n, batch_size):
        # Inputs
        input_words = input_data[i:i+batch_size]
        batch_size = input_words.shape[0]
        encoded_hidden_cell_states = encoder_model.predict(input_words)

        target_sentence = np.zeros((batch_size, 1, num_decoder_tokens))
        target_sentence[:, 0, target_char_index["\t"]] = 1.0
        target_sentence = np.argmax(target_sentence, axis=2)

        decoded_words = [""]*batch_size
        for i in range(max_decoder_seq_length):
            output_tokens, state, attn = decoder_model.predict([target_sentence] + [encoded_hidden_cell_states])

            sampled_char_index = np.argmax(output_tokens[:, -1, :], axis=1)

            target_sentence = np.zeros((batch_size, 1, num_decoder_tokens))

            for j, ch_index in enumerate(sampled_char_index):
                decoded_words[j] += reverse_target_char_index[ch_index]
                target_sentence[j, 0, ch_index] = 1.0

            target_sentence = np.argmax(target_sentence, axis=2)

            encoded_hidden_cell_states[1] = [state]

        decoded_words = [word[:word.find("\n")] for word in decoded_words]
        predicted = predicted + decoded_words
        
      
    actual_words = [word[1:-1] for word in target_words]
    # Calculate validation accuracy
    accuracy = np.mean(np.array(predicted) == np.array(actual_words))
    return accuracy,actual_words,predicted


In [16]:
Validation_accuracy,val_actual_word,val_predicted = calculate_accuracy(encoder_model,decoder_model,val_encoder_input_data,val_target_words)
Test_accuracy,test_actual_word,test_predicted = calculate_accuracy(encoder_model,decoder_model,test_encoder_input_data,test_target_words)
print("Validation_accuracy",Validation_accuracy * 100)
print("Test_accuracy",Test_accuracy*100)

Validation_accuracy 57.21422274463709
Test_accuracy 55.72954778281868


In [ ]:
keras.utils.plot_model(model)

In [ ]:
wandb.init(entity="swe-rana",project="CS6910Assignment3")
wandb.log({"Test accuracy" : Test_accuracy})

In [ ]:
data= pd.DataFrame({"Input Words":test_input_words,"Target words": test_actual_word,"Predicted_words": test_predicted})
data.to_csv("Test_Data_Predictions_with_attention.csv", index=False)


In [ ]:
def decode_best_model(input_word,encoder_model,decoder_model):
    # Encode the input string
    encoded_hidden_cell_states = encoder_model.predict(input_word)

    target_sentence = np.zeros((1, 1, num_decoder_tokens))
    # Set the first character to "tab" as it is the start of sequence character
    target_sentence[0, 0, target_char_index["\t"]] = 1.0
    target_sentence = np.argmax(target_sentence, axis=2)
    stop = False
    decoded_word = ""
    attention = []
    while not stop:
        output_tokens, h, attn = decoder_model.predict([target_sentence] + encoded_hidden_cell_states)
        attention.append(attn)

        sampled_char_index = np.argmax(output_tokens[0, -1, :])

        sampled_char = reverse_target_char_index[sampled_char_index]
        decoded_word += reverse_target_char_index[sampled_char_index]

        if sampled_char == "\n" or len(decoded_word) >= max_decoder_seq_length:
            stop = True

        # Update the target sequence which goes back as input to the decoder.
        target_sentence = np.zeros((1, 1, num_decoder_tokens))
        target_sentence[0, 0, sampled_char_index] = 1.0
        target_sentence = np.argmax(target_sentence, axis=2)

        # Update the hidden state and cell state 
        encoded_hidden_cell_states[1] = [h]

    return decoded_word,attention

**Plotting Heatmaps**

In [ ]:
target_word = []
input_word = []
predicted_word = []

random_indices = [np.random.randint(0, num_test) for i in range(9)]

fig = plt.figure(figsize=(10,10))

index=1
att = []
for i in random_indices:
    inp = test_encoder_input_data[i:i+1]
    decoded_word,attention = decode_best_model(inp,encoder_model,decoder_model)

    attention = np.array(attention)
    plt.subplot(3,3,index)
    plt.imshow(attention.reshape(attention.shape[0],attention.shape[-1])[:,:len(decoded_word)],cmap="magma")

    plt.yticks(range(attention.shape[0]),decoded_word,fontproperties=FontProperties(fname = '/content/Latha2.ttf'))
    plt.xticks(range(attention.shape[0]),test_input_words[i])

    input_word.append(test_input_words[i])
    predicted_word.append(decoded_word.strip("\n"))
    att.append(attention.reshape(attention.shape[0],attention.shape[-1])[:,:len(decoded_word)])
    target_word.append(test_target_words[i][1:-1])
    index+=1
plt.show()

df_train = pd.DataFrame({"Input": input_word, "Target" : target_word, "predicted":predicted_word})


In [ ]:
df_train.to_csv("gif.csv",index=False)

In [ ]:
import warnings
import cv2 as cv
from moviepy.editor import ImageSequenceClip



for num in range(9):
  text = df_train.iloc[num,2]
  sequence =[]

  data = df_train.iloc[num,0]
  warnings.filterwarnings('ignore')
  for i in range(len(text)):
    plt.text(0.8,0.8,text,fontproperties=FontProperties(fname = '/content/Latha2.ttf'),fontsize = 30)
    plt.text(0.8,0.8,text[i],fontproperties=FontProperties(fname = '/content/Latha2.ttf'),fontsize = 30)
    plt.axis('off')

    alphas = att[num][i]
    for j in range(len(data)):
      t=plt.text(0.5*(j/6),0.5,data[j],fontsize=30)
      #print(alphas[j])
      t.set_bbox(dict(facecolor='blue', alpha=alphas[j], edgecolor='blue'))
    plt.savefig('saved_img.png')
    plt.show()
    sequence.append(cv.cvtColor(cv.imread('save_gif.png'), cv.COLOR_BGR2RGB ))  

  gif = ImageSequenceClip(sequence, fps=5)
  gif.write_gif('conn' + str(num)+'.gif', fps=5)